<a href="https://colab.research.google.com/github/davidsolow/med-abbrev-mystery/blob/base-bert/MIMIC_mortality_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import re
import textwrap

from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizer, TFBertModel

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
total_abbreviations = pd.read_csv("drive/MyDrive/266Project/total_abbreviations.csv")

In [4]:
mimic_train = pd.read_csv("drive/MyDrive/266Project/medal_mimic_subset/train.csv")
mimic_validation = pd.read_csv("drive/MyDrive/266Project/medal_mimic_subset/valid.csv")
mimic_test = pd.read_csv("drive/MyDrive/266Project/medal_mimic_subset/test.csv")

In [5]:
print(mimic_train.TEXT[200_000])

npn :  0700 - 1900
nkda

full code

pt with pna admitted to mcu for further monitoring from floor when found with a pao2 of 75 on 2l nc . 

neuro :  pt is alert and oriented times three .  follows commands ,  c / o slight lower back pain and headache .  pt recieved tylenol times two .  mae .  oob to commode .  walked with pt . 

cv :  hr st .  100 - 140  '  s with activity .  md  '  s aware .  fluid bolus given times two .  slight le edema .  ppp bilaterally . 

resp :  ls coarse throughout but improving .  sating 88 - 100 %  on 2 l nc .  rr 20 - 31 .  denies sob .  neb times one . 

gi / gu :  bs active .  pt had two episodes of diarrhea .  stool sample collected .  hemeoccult positive .  pt has been voiding in bsc .  pt urine is pink / red ,  orginally thought it was from her menses ,  however ,  pt has not had any blood on Last Name  ( un )   pad .  pt had ua sent and md  '  s are aware .  pt hct has remained stable . 

endo :  on standing levemir .  insulin in med room .  humalog s

In [6]:
model_checkpoint = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
abbreviations = set(total_abbreviations.abbreviation)

def has_any_abbreviation(text):
    words = set(text.split())
    found = words.intersection(abbreviations)
    return bool(found)

mimic_train_subset = mimic_train.loc[mimic_train.TEXT.apply(has_any_abbreviation)]
mimic_validation_subset = mimic_validation.loc[mimic_validation.TEXT.apply(has_any_abbreviation)]
mimic_test_subset = mimic_test.loc[mimic_test.TEXT.apply(has_any_abbreviation)]

In [8]:
print(f"Train: {len(mimic_train_subset)} out of {len(mimic_train)}")
print(f"Validation: {len(mimic_validation_subset)} out of {len(mimic_validation)}")
print(f"Test: {len(mimic_test_subset)} out of {len(mimic_test)}")

Train: 201609 out of 204805
Validation: 27581 out of 28017
Test: 43030 out of 43717


In [9]:
mimic_no_abbreviation = mimic_train.loc[~mimic_train.TEXT.apply(has_any_abbreviation)]
mimic_no_abbreviation.head()

,HADM_ID,SUBJECT_ID,TEXT,HOSPITAL_EXPIRE_FLAG
75,116532.0,29487,Atrial fibrillation ( Afib ) \n Assessment ...,1.0
330,147435.0,29219,"Respiratory failure , chronic\n Assessment ...",1.0
432,116532.0,29487,Atrial fibrillation ( Afib ) \n Assessment ...,1.0
435,116532.0,29487,Impaired Skin Integrity\n Assessment : \n ...,1.0
517,167790.0,11288,"Respiration / Gas Exchange , Impaired\n A...",1.0


In [10]:
mimic_train_subset.SUBJECT_ID.nunique()

26162

In [11]:
MAX_LENGTH = 512

def head_and_tail(tokens, total_size=MAX_LENGTH):
    """Build an array of size <total_size> using the head and tail ends of the input array"""
    half = int(total_size / 2)
    if len(tokens) <= total_size:
        return tokens
    head = tokens[:half]
    tail = tokens[-half:]
    return np.concatenate([head, tail])


def tokenize(texts):
    """
    Tokenize an array of text inputs for bert, and take the head and tail of
    each set of bert inputs. Returns a tuple of three arrays: input_ids, token_type_ids,
    and attention_mask.
    """
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []

    for text in texts:
        bert_output = bert_tokenizer(
            text, padding="max_length", return_tensors="tf", max_length=MAX_LENGTH
        )
        input_ids = bert_output["input_ids"][0]
        token_type_ids = bert_output["token_type_ids"][0]
        attention_mask = bert_output["attention_mask"][0]

        input_ids_list.append(head_and_tail(input_ids))
        token_type_ids_list.append(head_and_tail(token_type_ids))
        attention_mask_list.append(head_and_tail(attention_mask))

    return np.array(input_ids_list), np.array(token_type_ids_list), np.array(attention_mask_list)


# tester = np.array([1,2,3,4,5,6,7,8,9,0])
# print(head_and_tail(tester, 4))
examples = mimic_train.TEXT[0:3].tolist()
input_ids, token_type_ids, attention_mask = tokenize(examples)

print(len(input_ids[2]))
print(examples[2])
print(bert_tokenizer.decode(input_ids[2]))




512
Sepsis ,  Severe  ( with organ dysfunction ) 
   Assessment : 
   Remains hypotensive on levophed and vasopressin gtt
s ,   MAP  > 60 ,  CVP
   8 - 13 ,  Lactic Acid down to 0 . 9
   Remains unresponsive off sedation meds since 1 - 27 AM . 
   Urine output diminished to  < 50cc / hr . 
   Afebrile . 
   HCT 25 . 3 ,  thrombocytopenic
   Action : 
   Titrating levophed gtt to keep MAP  > 60
   Received one unit PRBC
   Received 500cc NS bolus for UO  < 50cc / hr . 
   Remains on vasopressin gtt . 
   Head CT done 1 - 27 negative for bleed . 
   Response : 
   Levophed titrated down to 0 . 1 mcs / kg / min . 
   Improvement in CVP and urine output following PRBC
s and NS bolus
   HCT improved to 28 . 6 post transfustion . 
   Plan : 
   Continue pressors to keep MAP  > 60 ,  wean levophed as tolerated . 
   Continue to monitor UO and CVP ,  fluid bolus as needed . 
   Cont to monitor labs . 
   Pts daughter  ( HCP )  called overnight and updated by RN ,  will be into
   visit today .

In [12]:
#tokenizing subsets

mimic_train_subset_inputs = tokenize(mimic_train_subset.TEXT.tolist())
mimic_validation_subset_inputs = tokenize(mimic_validation_subset.TEXT.tolist())
mimic_test_subset_inputs = tokenize(mimic_test_subset.TEXT.tolist())

In [13]:
mimic_train_subset_labels = mimic_train_subset.HOSPITAL_EXPIRE_FLAG.to_numpy(np.int32)
mimic_validation_subset_labels = mimic_validation_subset.HOSPITAL_EXPIRE_FLAG.to_numpy(np.int32)
mimic_test_subset_labels = mimic_test_subset.HOSPITAL_EXPIRE_FLAG.to_numpy(np.int32)

In [15]:
def create_bert_base_model(
    bert_base_model,
    max_sequence_length=MAX_LENGTH,
    hidden_size=100,
    dropout=0.3,
    learning_rate=0.00005
):
    """
    Build a simple classification model with BERT. Use the pooler Token output for classification purposes.
    """
    bert_base_model.trainable = True

    input_ids = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_base_model(bert_inputs)
    pooler_token = bert_out[1]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)


    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                 metrics='accuracy')

    return classification_model

In [16]:
bert_model = TFBertModel.from_pretrained(model_checkpoint)
baseline_bert_model = create_bert_base_model(bert_model)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [19]:
checkpoint = ModelCheckpoint(filepath='drive/MyDrive/266Project/basebert_mimic_baseline_checkpoints/model_{epoch:02d}.hdf5',
                             save_weights_only=True,
                             save_best_only=False,
                             monitor='val_loss',
                             mode='auto',
                             save_freq='epoch',
                             verbose=1)

In [20]:
baseline_history = baseline_bert_model.fit(
    mimic_train_subset_inputs,
    mimic_train_subset_labels,
    epochs=5,
    batch_size=32,
    validation_data=(mimic_validation_subset_inputs, mimic_validation_subset_labels),
    callbacks=[checkpoint]
)

Epoch 1/5
6301/6301 [==============================] - ETA: 0s - loss: 0.4130 - accuracy: 0.8122
Epoch 1: saving model to drive/MyDrive/266Project/basebert_mimic_baseline_checkpoints/model_01.hdf5
6301/6301 [==============================] - 5443s 853ms/step - loss: 0.4130 - accuracy: 0.8122 - val_loss: 0.4078 - val_accuracy: 0.8159
Epoch 2/5
6301/6301 [==============================] - ETA: 0s - loss: 0.3398 - accuracy: 0.8541
Epoch 2: saving model to drive/MyDrive/266Project/basebert_mimic_baseline_checkpoints/model_02.hdf5
6301/6301 [==============================] - 5365s 851ms/step - loss: 0.3398 - accuracy: 0.8541 - val_loss: 0.3970 - val_accuracy: 0.8254
Epoch 3/5
6301/6301 [==============================] - ETA: 0s - loss: 0.3090 - accuracy: 0.8691
Epoch 3: saving model to drive/MyDrive/266Project/basebert_mimic_baseline_checkpoints/model_03.hdf5
6301/6301 [==============================] - 5364s 851ms/step - loss: 0.3090 - accuracy: 0.8691 - val_loss: 0.4289 - val_accuracy: 0.

KeyboardInterrupt: 

In [21]:
checkpoint_path = 'drive/MyDrive/266Project/basebert_mimic_baseline_checkpoints/model_02.hdf5'
baseline_bert_model.load_weights(checkpoint_path)

In [22]:
loss, accuracy = baseline_bert_model.evaluate(mimic_test_subset_inputs, mimic_test_subset_labels, verbose=1)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

1345/1345 [==============================] - 368s 273ms/step - loss: 0.3738 - accuracy: 0.8355
Test loss: 0.3738192617893219
Test accuracy: 0.835486888885498


In [23]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [27]:
#calculate F1, precision, recall
y_pred = baseline_bert_model.predict(mimic_test_subset_inputs)


1345/1345 [==============================] - 362s 269ms/step


In [30]:
y_pred[0]

array([0.9086342], dtype=float32)

In [28]:
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = mimic_test_subset_labels

f1 = f1_score(y_true, y_pred_classes, average='macro')
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')

print(f'F1: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

#classification report
report = classification_report(y_true, y_pred_classes, target_names=['Did not die in hospital', 'Died in hospital'])
print(report)

F1: 0.32481837724184465
Precision: 0.2405414826864978
Recall: 0.5
                         precision    recall  f1-score   support

Did not die in hospital       0.48      1.00      0.65     20701
       Died in hospital       0.00      0.00      0.00     22329

               accuracy                           0.48     43030
              macro avg       0.24      0.50      0.32     43030
           weighted avg       0.23      0.48      0.31     43030



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13